In [1]:
import pyqstrat as pq
import h5py

def create_data():
    ohlcs = []

    with h5py.File('SPX.hdf5', 'r') as f:
        dates = np.arange(np.datetime64('2023-01-01'), np.datetime64('2023-01-10'))
        for date in dates:
            _date = pd.to_datetime(date).to_pydatetime()
            key = f'D_{_date:%Y_%m_%d}'
            if key not in f: continue
            _df = pq.hdf5_to_df('/Users/sal/Developer/ambrus/reports/tail_options/SPX.hdf5', key)
            df = _df.copy()
            df['expiry'] = pd.to_datetime(df.okey_yr * 10000 + df.okey_mn * 100 + df.okey_dy, format='%Y%m%d')
            df = df[['timestamp', 'okey_cp', 'expiry', 'okey_xx', 'prtPrice', 'prtSize', 'prtVolume', 'uBid', 'uAsk', 'prtIv', 'prtDe']]
            df['umid'] = 0.5 * (df.uBid + df.uAsk)
            del df['uBid']
            del df['uAsk']
            df.columns = ['timestamp', 'put_call', 'expiry', 'strike', 'prc', 'qty', 'v', 'iv', 'delta', 'umid']
            df['symbol'] = df.put_call.str[:1] + '-' + df.strike.astype(int).astype(str) + '-' + df.expiry.astype(str)
            df['timestamp'] = pd.to_datetime(df.timestamp)
            ohlc = df.set_index('timestamp').groupby(
                ['symbol'], as_index=False).resample('5min').agg(
                {'symbol': 'last', 
                 'prc': ['first', 'min', 'max', 'last'], 
                 'umid': 'last', 
                 'v': 'sum',
                 'iv': 'last',
                 'delta': 'last'})
            ohlc.reset_index(inplace=True)
            ohlc.columns = ['dummy', 'timestamp', 'symbol', 'o', 'h', 'l', 'c', 'umid', 'v', 'iv', 'delta']
            ohlc = ohlc[np.isfinite(ohlc.c)]
            ohlc = ohlc[['timestamp', 'symbol', 'o', 'h', 'l', 'c', 'v', 'umid', 'iv', 'delta']]
            #ohlc = ohlc[['timestamp', 'open', 'high', 'low', 'close']]
            ohlcs.append(ohlc)

    pd.concat(ohlcs).to_csv('support/spx_options.csv.gz', index=False)
    pd.read_csv('support/spx_options.csv.gz')
